# Classification supervisée

In [1]:
from src.supervised_classification import *

## Chargement des données

In [2]:
# 1. Chargement des données
input_file = "articles_with_clusters.pkl"
df = load_processed_data(input_file)

Données chargées avec succès. Shape: (23702, 13)


## Préparation des données

In [3]:
# Application de la fonction pour créer la colonne 'class'
df['class'] = df['venue'].apply(assign_class_from_venue)

# Vérification de la répartition des classes
print(df['class'].value_counts().sort_index())

df = prepare_text_column(df)

class
1      742
2      875
3      195
4      746
5     2738
6      409
7      298
8    17699
Name: count, dtype: int64


## Exploration de la distribution des classes

In [4]:
# 3. Exploration de la distribution des classes
fig = explore_class_distribution(df)
fig.show()

Nombre total d'articles classifiés: 23702
Classe 1 (Artificial Intelligence): 742 articles (3.1%)
Classe 2 (Data Science): 875 articles (3.7%)
Classe 3 (Interface): 195 articles (0.8%)
Classe 4 (Computer Vision): 746 articles (3.1%)
Classe 5 (Network): 2738 articles (11.6%)
Classe 6 (Theoretical CS): 409 articles (1.7%)
Classe 7 (Specific Applications): 298 articles (1.3%)
Classe 8 (Other): 17699 articles (74.7%)


## Extraction des features textuelles

In [5]:
# 4. Extraction des features textuelles
text_extractor = TextFeatureExtractor(df, 'combined_text', 'class')
text_features = text_extractor.extract_tfidf_features(min_df=5, max_df=0.95, ngram_range=(1, 2))

# Visualisation de l'importance des features par classe
fig_text_importance = text_extractor.visualize_feature_importance(top_n=20)
fig_text_importance.show()

Extraction des features TF-IDF...
Extraction terminée en 23.07 secondes.
Nombre de features: 42300
Dimensions de la matrice: (23702, 42300)


## Extraction des features structurelles

In [6]:
# 5. Extraction des features structurelles
# Identification des colonnes structurelles
structural_columns = {
    'authors': 'authors',
    'references': 'references',
    'venue': 'venue'
}

graph_extractor = GraphFeatureExtractor(df)
graph = graph_extractor.build_graph(structural_columns)
graph_features = graph_extractor.extract_graph_features()

# Visualisation des features structurelles
fig_graph_features = graph_extractor.visualize_graph_features()
fig_graph_features.show()

Construction du graphe à partir des colonnes structurelles...


Ajout des arêtes: 100%|██████████| 1287864/1287864 [00:01<00:00, 753857.86it/s]


Graphe construit: 23702 nœuds, 1287864 arêtes
Extraction des features structurelles...
Extraction des features de degré...
Extraction des features de centralité...
Extraction des features de clustering...
Extraction des features de PageRank...
Extraction terminée. Dimensions de la matrice: (23702, 6)


## Combinaison des features et classification

In [9]:
# 6. Combinaison des features et classification
classifier = DocumentClassifier(df, text_features, graph_features, 'class')
X_combined = classifier.combine_features(standardize_graph=True)

# Division des données
classifier.split_data(test_size=0.2)

# Entraînement et évaluation de plusieurs modèles
models_config = [
    {'type': 'svm', 'params': {'C': 1.0, 'kernel': 'linear', 'random_state': 42}},
    {'type': 'rf', 'params': {'n_estimators': 100, 'max_depth': 10, 'random_state': 42}},
    {'type': 'lr', 'params': {'C': 1.0, 'max_iter': 1000, 'random_state': 42}}
]

results = classifier.train_and_evaluate_multiple_models(models_config)

# Visualisation de la matrice de confusion pour le meilleur modèle
best_model_full_name = max(results.items(), key=lambda x: x[1]['accuracy'])[0]

# Affichage de la matrice de confusion avec le nom complet du modèle
fig_confusion = classifier.visualize_confusion_matrix(model_type=best_model_full_name)
fig_confusion.show()

# Mapping des noms complets vers les identifiants courts
model_name_mapping = {
    'SVC': 'svm',
    'RandomForestClassifier': 'rf',
    'LogisticRegression': 'lr'
}

# Conversion du nom complet en identifiant court pour l'optimisation des hyperparamètres
best_model_short_name = model_name_mapping.get(best_model_full_name)

# Optimisation des hyperparamètres du meilleur modèle avec l'identifiant court
if best_model_short_name:
    opt_results = classifier.optimize_hyperparameters(model_type=best_model_short_name)
else:
    print(f"Impossible de mapper le modèle {best_model_full_name} à un identifiant court reconnu.")

# Visualisation de l'importance des features (utilisez le nom complet pour accéder aux résultats stockés)
if best_model_short_name in ['rf', 'gb']:
    fig_importance = classifier.visualize_feature_importance(model_type=best_model_short_name)
    fig_importance.show()

Combinaison des features textuelles et structurelles...
Features combinées. Dimensions de la matrice: (23702, 42306)
Division des données (test_size=0.2)...
Ensemble d'entraînement: (18961, 42306)
Ensemble de test: (4741, 42306)
Entraînement d'un modèle svm...
Entraînement terminé en 827.81 secondes.
Évaluation du modèle SVC...
Exactitude (accuracy): 0.7524

Rapport de classification:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       148
           2       0.75      0.02      0.03       175
           3       1.00      0.03      0.05        39
           4       0.72      0.12      0.21       149
           5       0.71      0.02      0.04       548
           6       0.83      0.06      0.11        82
           7       0.00      0.00      0.00        60
           8       0.75      1.00      0.86      3540

    accuracy                           0.75      4741
   macro avg       0.60      0.16      0.16      4741
weighted avg    

Optimisation des hyperparamètres pour le modèle svm...
Fitting 5 folds for each of 18 candidates, totalling 90 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.
